In [ ]:
!pip install pandas numpy scikit-learn tensorflow

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Load the dataset
df = pd.read_csv("./data.csv")

# Encode 'Source' and 'Destination' if they are categorical
df['Source'] = df['Source'].astype('category').cat.codes
df['Destination'] = df['Destination'].astype('category').cat.codes

# Normalize the data
scaler = MinMaxScaler()
df[['Source', 'Destination', 'Avg_Delay(ms)']] = scaler.fit_transform(df[['Source', 'Destination', 'Avg_Delay(ms)']])

# Prepare data for LSTM
X = df[['Source', 'Destination']].values
y = df['Avg_Delay(ms)'].values
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),
          verbose=1, callbacks=[early_stopping])

# Predict the values
y_pred = model.predict(X_test)*

# Calculate and print metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')


# Define an acceptable error tolerance for custom accuracy
tolerance = 0.01  # Adjust this based on the scale of your data
accurate_predictions = np.abs(y_test - y_pred.flatten()) <= tolerance
accuracy = np.mean(accurate_predictions) * 100

print(f'Custom Accuracy: {accuracy:.2f}%')


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - loss: 4.1759e-04 - val_loss: 3.0433e-05
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.4299e-04 - val_loss: 3.6980e-05
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.0074e-04 - val_loss: 1.3923e-05
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0014 - val_loss: 8.7277e-06
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.5895e-04 - val_loss: 4.7195e-05
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0036 - val_loss: 1.3101e-05
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0036 - val_loss: 5.8013e-06
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.1866e-04 - val_loss: 6.9060e-05
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0020 - val_loss: 5.4072e-06
Epoch 10/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0049 - val_loss: 5.4296e-06
Epoch 11/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0049 - val_loss: 5.6967e-06
Epoc

In [ ]:
# Define the custom links
custom_links = np.array([
    [4,2], [2,3], [10,11], [11, 1],
    [1, 6]
])

## Add a dummy column for 'Avg_Delay(ms)' (e.g., zeros)
custom_links_with_dummy = np.hstack((custom_links, np.zeros((custom_links.shape[0], 1))))

# Normalize the custom links using the same scaler
custom_links_normalized = scaler.transform(custom_links_with_dummy)

# Extract only the normalized 'Source' and 'Destination' columns
custom_links_normalized = custom_links_normalized[:, :2]

# Reshape the data to match the input shape for the LSTM model
custom_links_reshaped = custom_links_normalized.reshape((custom_links_normalized.shape[0], 1, custom_links_normalized.shape[1]))


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
# Predict delays for custom links
custom_predictions = model.predict(custom_links_reshaped)

# Inverse transform the predictions to get actual delay values
custom_delays = scaler.inverse_transform(
    np.hstack((custom_links, custom_predictions.reshape(-1, 1)))
)[:, -1]

# Display the predicted delays
for link, delay in zip(custom_links, custom_delays):
    print(f'Link {tuple(link)}: Predicted Delay = {delay:.6f} ms')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Link (3, 8): Predicted Delay = 0.000230 ms
Link (8, 10): Predicted Delay = 0.000254 ms
Link (10, 11): Predicted Delay = 0.000262 ms
Link (11, 1): Predicted Delay = 0.000287 ms
Link (1, 6): Predicted Delay = 0.000221 ms
